In [1]:
def data_prepare():
    boston = load_boston()
    boston_feature = pd.DataFrame(boston.data, columns=boston.feature_names)
    boston_label = pd.Series(boston.target).to_frame("TARGET")

    boston = pd.concat([boston_label, boston_feature], axis=1)

    return boston

data = data_prepare()
print(data.shape)
(506, 14)
# 查看现有特征分布
plot_dist(data, data.columns)

NameError: name 'load_boston' is not defined